# Accessing TTree data with PyROOT (and others)

Very likely, most of your data are in ROOT TTrees. Here are various ways of getting the data out.

In [ ]:
import ROOT

In [ ]:
# Note: this is TFile::Open(...)
file = ROOT.TFile.Open("http://scikit-hep.org/uproot/examples/HZZ.root")
tree = file.Get("events")

In [ ]:
for event in tree:
    print(event.MET_px, event.MET_py)

That worked, but it would be uncomfortable on large datasets because expressions like `event.MET_px` does a lot of processing _in each event_ to produce a number. The same reasons to use Numpy over Python for loops applies here (to an even greater degree).

Fortunately, ROOT now has methods to access TTree data as Numpy arrays.

In [ ]:
tree.AsMatrix(["MET_px", "MET_py"])

The labels (`MET_px`, `MET_py`) are missing, but here's a way to get them:

In [ ]:
data, labels = tree.AsMatrix(["MET_px", "MET_py"], return_labels=True)
labels

And use them:

In [ ]:
recarray = data.reshape(-1).view([(x, data.dtype) for x in labels])
recarray

In [ ]:
recarray["MET_px"]

In [ ]:
recarray["MET_py"]

## Nested structure

Not all data have one value per event/entry.

In [ ]:
for event in tree:
    print([x for x in event.Muon_Px])

These kinds of data don't have a Numpy analogue, so `AsMatrix` can't read them.

In [ ]:
tree.AsMatrix(["Muon_Px"])

ROOT doesn't (yet) have a method for this, but two Python packages, root_numpy and uproot, do (check a lecture about uproot this afternoon with Mason and Emma).

## Arbitrary objects

PyROOT should be able to read any kind of object, as it's a front end for ROOT.

In [ ]:
file2 = ROOT.TFile.Open("http://scikit-hep.org/uproot/examples/HZZ-objects.root")
tree3 = file2.Get("events")

In [ ]:
for event in tree3:
    print(event.muonp4, [x.Pt() for x in event.muonp4])

## Summary

Which method you choose depends on accessibility of ROOT, what types of data you need to read, and the scale of your dataset.

<table style="font-size: 22pt; margin-top: 50px">
    <tr style="font-weight: bold"><td>Method</td><td>Relationship to ROOT</td><td>Data types</td><td>Performance</td></tr>
    <tr><td>PyROOT for loop</td><td>part of ROOT</td><td>any</td><td>slow</td></tr>
    <tr><td>PyROOT AsMatrix</td><td>part of ROOT</td><td>one number per entry</td><td>fast</td></tr>
</table>